In [1]:

import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split

# Home directory
home_path = r'C:\Users\c_fre\Learn.co\Module_4_Project\Image-Classification\chest_xray'

# Create train and validation directories
train_path = os.path.join(home_path,'train')
os.mkdir(train_path)
val_path = os.path.join(home_path,'val')
os.mkdir(val_path)

# Create sub-directories
normal_train = os.path.join(home_path + r'/train','NORMAL')
os.mkdir(normal_train)

pneumonia_train = os.path.join(home_path + r'/train','PNEUMONIA')
os.mkdir(pneumonia_train)

normal_val = os.path.join(home_path + r'/val','NORMAL')
os.mkdir(normal_val)

pneumonia_val = os.path.join(home_path + r'/val','PNEUMONIA')
os.mkdir(pneumonia_val)


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\c_fre\\Learn.co\\Module_4_Project\\Image-Classification\\chest_xray\\train'

In [2]:
# ImageDataGenerator flow_from_dataframe

df_train = home_path + r'/Users/c_fre/Learn.co/Module_4_Project/Image-Classification/chest_xray/train/'
df_train['pneumonia_or_not'] = df_train['pneumonia_or_not'].astype('str') # requires target in string format

train_generator_df = datagen.flow_from_dataframe(dataframe=df_train, 
                                              directory=home_path+'/images/',
                                              x_col="image_names", 
                                              y_col="pneumonia_or_not", 
                                              class_mode="binary", 
                                              target_size=(200, 200), 
                                              batch_size=1,
                                              rescale=1.0/255,
                                              seed=2020)
# plotting images
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15,15))

for i in range(4):

  # convert to unsigned integers for plotting
  image = next(train_generator_df)[0].astype('uint8')

  # changing size from (1, 200, 200, 3) to (200, 200, 3) for plotting the image
  image = np.squeeze(image)

  # plot raw pixel data
  ax[i].imshow(image)
  ax[i].axis('off')

TypeError: string indices must be integers

## Table Of Contents
<font size=3rem>
    
0 -**[ INTRO](#INTRODUCTION)<br>**
1 -**[ OBTAIN](#OBTAIN)**<br>
2 -**[ PREPROCESSING](#PREPROCESSING)**<br>
4 -**[ MODEL](#MODEL)**<br>
5 -**[ INTERPRET](#INTERPRET)**<br>
6 -**[ CONCLUSIONS & RECCOMENDATIONS](#Conclusions-&-Recommendations)<br>**
</font>
___

# INTRODUCTION

- Student: Cody Freese
- Pace: Self Paced
- Jeff Herman

- Pneumonia Chest Xray Image Classification
    - Objective
        - First I need to decide on:
            - Accuracy
            - Precision
            - Recall
            - F1 Score
        - Priority
            - (1) Do not want to tell a sick person they are healthy
            
            - (2) Accurately identify pneumonia
            - (3) Tell healthy person they are sick

## Import Tools

In [10]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
import cv2

# OBTAIN

In [3]:
#Home Directory
home_path = r'C:\Users\c_fre\Learn.co\Module_4_Project\Image-Classification\chest_xray'

#Train Directory
train_path = os.path.join(home_path,'train')

#Validation Directory
val_path = os.path.join(home_path,'val')

#Test Directory
test_path = os.path.join(home_path,'test')

#Train Subdirectories
normal_train_path = os.path.join(train_path,'NORMAL')

pneumonia_train_path = os.path.join(train_path,'PNEUMONIA')

#Validation Subdirectories
normal_val_path = os.path.join(val_path,'NORMAL')

pneumonia_val_path = os.path.join(val_path,'PNEUMONIA')

#Test Subdirectories

normal_test_path = os.path.join(test_path,'NORMAL')

pneumonia_test_path = os.path.join(test_path,'PNEUMONIA')

In [4]:
#Observe Contents
print('Directory Contents:',os.listdir(home_path))
print('train Contents:',os.listdir(train_path))
print('val Contents:',os.listdir(val_path))
print('test Contents:',os.listdir(test_path))

Directory Contents: ['test', 'train', 'val']
train Contents: ['NORMAL', 'PNEUMONIA']
val Contents: ['NORMAL', 'PNEUMONIA']
test Contents: ['NORMAL', 'PNEUMONIA']


In [5]:
#Cases in Training Sets
print("Number of cases in Training:",len(os.listdir(normal_train_path)) +len(os.listdir(pneumonia_train_path)))

print("Number of Pneumonia cases in Training:",len(os.listdir(pneumonia_train_path)))

print("Number of Normal cases in Training:",len(os.listdir(normal_train_path)))

Number of cases in Training: 5216
Number of Pneumonia cases in Training: 3875
Number of Normal cases in Training: 1341


In [6]:
# Cases in Validation Sets
print("Number of cases in Validation:",len(os.listdir(normal_val_path)) +len(os.listdir(pneumonia_val_path)))

print("Number of Pneumonia cases in Validation:",len(os.listdir(pneumonia_val_path)))

print("Number of Normal cases in Validation:",len(os.listdir(normal_val_path)))

Number of cases in Validation: 16
Number of Pneumonia cases in Validation: 8
Number of Normal cases in Validation: 8


In [7]:
#Cases in Test Sets
print("Number of cases in Test:",len(os.listdir(normal_test_path)) +len(os.listdir(pneumonia_test_path)))

print("Number of Pneumonia cases in Test:",len(os.listdir(pneumonia_test_path)))

print("Number of Normal cases in Test:",len(os.listdir(normal_test_path)))

Number of cases in Test: 624
Number of Pneumonia cases in Test: 390
Number of Normal cases in Test: 234


In [8]:
# #Total number of Normal Cases
print("Total number of Normal cases:",
      len(os.listdir(normal_test_path))+
      len(os.listdir(normal_train_path))+
      len(os.listdir(normal_val_path)))

#Total number of Pneumonia Cases
print("Total number of Pneumonia cases:",
      len(os.listdir(pneumonia_test_path))+
      len(os.listdir(pneumonia_train_path))+
      len(os.listdir(pneumonia_val_path)))

Total number of Normal cases: 1583
Total number of Pneumonia cases: 4273


# PREPROCESSING

In [12]:
train_dataset = keras.preprocessing.image_dataset_from_directory(train_path,
                                                                labels='inferred',
                                                                label_mode='binary',
                                                                smart_resize=True)

val_dataset = keras.preprocessing.image_dataset_from_directory(val_path,
                                                               labels='inferred',
                                                               label_mode='binary',
                                                               smart_resize=True)

test_dataset = keras.preprocessing.image_dataset_from_directory(test_path,
                                                               labels='inferred',
                                                               label_mode='binary',
                                                               smart_resize=True)

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [9]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

<BatchDataset shapes: ((None, 256, 256, 3), (None, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 256, 256, 3), (None, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 256, 256, 3), (None, 1)), types: (tf.float32, tf.float32)>


In [21]:
# Test Image
img_path = '../C:/Users/c_fre/Learn.co/Module_4_Project/Image-Classification/chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg'

image = cv2.imread(img_path)

print(image)

None


In [14]:
fig=plt.figure(figsize=(15, 10))
columns = 4; rows = 5
for i in range(1, columns*rows +1):
    img = cv2.imread(#THIS NEEDS FIXING###[i])
    img = cv2.resize(img, (128, 128))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.axis(False)

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1bq9o88m\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


<Figure size 1080x720 with 0 Axes>

In [10]:
# print(f'Normal size:  {train_dataset.shape}')
# print(f'Pneumonia size:  {img_p.shape}')

In [11]:
epochs = 10
img_size = 150
batch_size = 32

In [1]:
#All below attempted to fit from ImageDataGenerator. Find way to make above fit with below

In [12]:
# ImageDataGenerator allows us to 'rotate' to analyze the image in more than 2D
train_datagen = ImageDataGenerator(rescale = 1./255, # transform pixeles from range 0-255 to range 0-1
                                   zoom_range = 0.3, # randomly zoom image 
                                   vertical_flip = True) # randomly flip images

# load a mini batch of images directly from the source folder then convert them into a vector of attributes
training_set = train_datagen.flow_from_directory(directory = home_path + '/'+'train',
                                                 batch_size = batch_size,
                                                 target_size = (img_size, img_size),
                                                 shuffle = True,
                                                 class_mode = 'binary')

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
# Test Data
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(directory = home_path + '/'+'test',
                                            batch_size = batch_size,
                                            target_size = (img_size, img_size),
                                            shuffle = True,
                                            class_mode = 'binary')

# MODELING

In [ ]:
model = Sequential()

In [ ]:
# 1st conv
model.add(Conv2D(filters = 16, kernel_size = (3, 3), input_shape = (img_size, img_size, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPool2D(pool_size = (2, 2)))

In [ ]:
# 2nd conv
model.add(SeparableConv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(SeparableConv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))

In [ ]:
# 3rd conv
model.add(SeparableConv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(SeparableConv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

In [ ]:
# 4th conv
model.add(SeparableConv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(SeparableConv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))

In [ ]:
# 5th
model.add(SeparableConv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(SeparableConv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))

In [ ]:
# Connected layer
model.add(Flatten())
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(rate = 0.7))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.3))

In [ ]:
# Output layer
model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
#Model Summary
model.summary()


In [ ]:
# Sequntial Compile
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Callback
checkpoint = ModelCheckpoint(filepath = 'Pneumonia.h5', save_best_only = True, save_weights_only = True)
lr_reduce = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 2, verbose = 2, mode = 'max')
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.1, patience = 1, mode = 'min')

In [ ]:
# Fit data to cnn model
cnn = model.fit_generator(training_set,
                          epochs = epochs,
                          steps_per_epoch = training_set.samples // batch_size,
                          validation_data = test_set,
                          validation_steps = test_set.samples // batch_size,
                          callbacks = [checkpoint, lr_reduce])   

In [ ]:
model.load_weights('./Pneumonia.h5')

In [ ]:
#Input Test
rand_img = np.random.randint(0, len(os.listdir(drct_path +'/'+ 'val/PNEUMONIA/')))
rand_img_path = drct_path + '/'+'val/PNEUMONIA/' + os.listdir(drct_path +'/'+ 'val/PNEUMONIA/')[rand_img]

img = plt.imread(rand_img_path)
img = cv2.resize(img, (img_size, img_size))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = model.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] >0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(f'Predicted : ' + prediction)
print ('Actual : Pnuemonia')

image = plt.imread(rand_img_path)
plt.imshow(image)
plt.title('Pneumonia')
plt.imshow(image, cmap = 'gray')

In [ ]:
# Metrics
tn, fp, fn, tp = mat.ravel()
print('Test Metrics:')
precision = tp / (tp + fp) * 100
recall = tp / (tp + fn) * 100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

In [ ]:
print('Train Metric:')
print('Train accuracy: {}%'.format(np.round(cnn.history['accuracy'][-1]*100, 2)))